In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# 向量投影函数
# 计算向量 a 在向量 b 上的投影
def vector_projection(a, b):

    b_norm_sq = np.dot(b, b)
    if b_norm_sq < 1e-8:
        raise ValueError("基向量 b 的模长不能为零")
    projection = np.dot(a, b) / b_norm_sq * b
    return projection

In [ ]:
class Hanging_rope_multinode:

    #绳子基本参数
    #pos[3:6]为1*6向量，lenth为float，mass为float，ea为float，gravity为float
    def __init__(self,pos,vel,length,mass,d,EA,C,node_number,gravity=9.81):
        #先初始化，再赋值
        self.pos = np.zeros((3, 2, node_number), dtype=float)
        self.length = np.zeros(node_number, dtype=float)
        self.mass = np.zeros(node_number, dtype=float)
        self.gravity = gravity
        self.ea = EA
        self.vel = np.zeros((3, 2, node_number), dtype=float)
        self.C = float(C)
        self.d = float(d)
        self.node_number = node_number
        #calculate node mass

        # 计算每个节点的质量
        self.node_mass = self.calculate_node_mass()
        #计算绳子受力
        self.tension_force = self.calculate_tension_force(EA,d,length, pos)
        self.global_force = self.calculate_global_force(mass, gravity)
        self.damping_force = self.calculate_damping_force(pos, vel, length, d, damping_coefficient=C)
        self.total_force = self.tension_force + self.global_force + self.damping_force
    
    def calculate_node_mass(self):
        node_mass = np.zeros(self.node_number, dtype=float)
        # 计算每个节点的质量
        for i in range(self.node_number):
            if i == 0 or i == self.node_number - 1:
                node_mass[i] = 0.5 * self.mass / (self.node_number-1)
            else:
                # 中间节点质量为总质量的平均分配
                node_mass[i] = self.mass / (self.node_number - 1)
        return node_mass
    
    #计算每一段的张力
    def calculate_tension_force(self):

        #计算缆绳所受的张力
        #sward为一个质点的方向向量，指向另一个点
        #pos0-2是上点坐标，pos3-5是下端点坐标
        #将力输出为一个3*3的矩阵

        #初始化参数
        pos = np.array((3,2,self.node_number),dtype=float)
        lenth = float(lenth)
        EA = float(EA)
        d = float(d)
        #第一个为xyz力，第二个为top或者end，第三个为节点数
        force = np.zeros((3*2*self.node_number),dtype=float)
        sward_before = np.zeros(3,dtype=float)
        lenth_now = float(0.0)
        sward = np.zeros(3,dtype=float)

        #计算缆绳实际长度
        for i in range(self.node_number - 1):
            sward_before += pos[i*3:i*3+3] - pos[(i+1)*3:(i+1)*3+3]
        sward_before= (pos[0:3] - pos[3:6])
        lenth_now = numpy.linalg.norm(sward_before)
        if lenth_now <= 1e-6:
            force = np.zeros(6, dtype=float)
            return force
        sward = sward_before / lenth_now
        #计算缆绳长度变化
        lenth_changerate = lenth_now - lenth
        #计算点所受张力
        #前三个为下端点受力，后三个为上端点受力
        if lenth_changerate >= 0: 
            force[3:6] =  1/4*numpy.pi*EA*d**2*lenth_changerate*sward
            force[0:3] = -1/4*numpy.pi*EA*d**2*lenth_changerate*sward
        else:
            force = 0
        return force